# First steps with Claude throught Bedrock

## The most simple approach

In [1]:
import os
import boto3
from dotenv import load_dotenv

In [2]:
load_dotenv() # load the environment variables

True

In [3]:
profile_name=os.getenv("PROFILE_NAME") #  I currently log in using SSO.

In [4]:
session = boto3.session.Session(profile_name=profile_name, region_name="us-west-2")

In [5]:
client = session.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" # this refer to inference profile ID

user_message = {
    "role": "user",
    "content": [
        {"text": "What's 1+1?"}
    ]
}

response = client.converse(
    modelId = model_id,
    messages = [user_message]
)

In [6]:
response["output"]["message"] 

{'role': 'assistant', 'content': [{'text': '1 + 1 = 2'}]}

## Multi-turn approach

In [7]:
client = session.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" # this refer to inference profile ID

def add_user_message(messages: list, text: str):
        
    user_message = {
        "role": "user",
        "content": [
            {"text": text}
        ]
    }

    messages.append(user_message)
    

def add_assistant_message(messages: list, text: str):
        
    assistant_message = {
        "role": "assistant",
        "content": [
            {"text": text}
        ]
    }

    messages.append(assistant_message) 

def chat(messages: list):
    response = client.converse(
        modelId = model_id,
        messages = messages
    )

    return response["output"]["message"]["content"][0]["text"]

In [8]:
# started with an empty list of messages where we maintaining the context
messages = []

# initial user question
add_user_message(messages, "What's 1+1?")

# pass the list of messages into chat
answer = chat(messages)

# take the answer from the assistant and added it into the message list  
add_assistant_message(messages, answer)

# followup question
add_user_message(messages, "added 3 more, what's the result?")

# chat again
answer = chat(messages)

answer

'If we add 3 more to the previous result of 2:\n\n2 + 3 = 5\n\nSo the result is 5.'

### Query Processing

In [9]:
messages=[]
client = session.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" # this refer to inference profile ID

def process_query(query: str):
    
    add_user_message(messages, query)
    answer = chat(messages)
    add_assistant_message(messages, answer)
    return answer
                

# Chat Loop

In [10]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query)
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [11]:
chat_loop()

Type your queries or 'quit' to exit.


Query:  1+1



Assistant: 1 + 1 = 2


Query:  add 3



Assistant: 2 + 3 = 5


Query:  quit


# System Prompt

## First Approach

In [45]:
from typing import Optional, Union

def add_user_message(messages: list, text: str):
        
    user_message = {
        "role": "user",
        "content": [
            {"text": text}
        ]
    }

    messages.append(user_message)
    

def add_assistant_message(messages: list, text: str):
        
    assistant_message = {
        "role": "assistant",
        "content": [
            {"text": text}
        ]
    }

    messages.append(assistant_message) 

def chat(messages: list, system_prompt:Optional[str] = None, temperature: Union[float, int] = 1.0, stop_sequences=["5"]):
   
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences":stop_sequences,
        },
    }

    if system_prompt:
        params["system"] = [{"text": system_prompt}]
            
    response = client.converse(**params)

    return response["output"]["message"]["content"][0]["text"]

In [46]:
messages=[]
client = session.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" # this refer to inference profile ID

def process_query(query: str, system_prompt:Optional[str] = None, temperature: Union[float, int] = 1.0):
    
    add_user_message(messages, query)
    answer = chat(messages, system_prompt, temperature)
    add_assistant_message(messages, answer)
    return answer
                

## Without System prompt

In [17]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query)
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [18]:
chat_loop()

Type your queries or 'quit' to exit.


Query:  Give a way to deploy a web page



Assistant: Here's a simple way to deploy a web page using **Netlify** (free and beginner-friendly):

## Method 1: Drag & Drop Deployment

### Steps:
1. **Prepare your files**
   - Create your HTML, CSS, and JavaScript files
   - Put them in a single folder
   - Make sure your main file is named `index.html`

2. **Go to Netlify**
   - Visit [netlify.com](https://netlify.com)
   - Sign up for a free account

3. **Deploy**
   - Drag and drop your folder directly onto the Netlify dashboard
   - Netlify will automatically deploy your site
   - You'll get a live URL instantly (e.g., `random-name-123.netlify.app`)

### Example file structure:
```
my-website/
├── index.html
├── style.css
├── script.js
└── images/
    └── logo.png
```

## Method 2: GitHub + Netlify (Recommended)

1. **Upload to GitHub**
   - Create a repository on GitHub
   - Upload your web files

2. **Connect to Netlify**
   - In Netlify, click "New site from Git"
   - Connect your GitHub repository
   - Auto-deploy on every

Query:  quit


## System prompt GCP expert

In [19]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query, system_prompt="You are a GCP expert")
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [20]:
chat_loop()

Type your queries or 'quit' to exit.


Query:  Give a way to deploy a web page



Assistant: Here's a simple way to deploy a web page using **Google Cloud Platform (GCP)**:

## Method 1: Google Cloud Storage (Static Website)

### Steps:

1. **Create a Storage Bucket**
   ```bash
   # Using gcloud CLI
   gsutil mb gs://your-website-bucket-name
   ```

2. **Upload your web files**
   ```bash
   gsutil cp -r ./your-website/* gs://your-website-bucket-name/
   ```

3. **Make bucket public and configure for web hosting**
   ```bash
   # Make bucket publicly readable
   gsutil iam ch allUsers:objectViewer gs://your-website-bucket-name

   # Set index and error pages
   gsutil web set -m index.html -e 404.html gs://your-website-bucket-name
   ```

4. **Access your site**
   - URL: `https://storage.googleapis.com/your-website-bucket-name/index.html`

## Method 2: Google App Engine (More Features)

### Steps:

1. **Create app.yaml file**
   ```yaml
   runtime: python39

   handlers:
   - url: /
     static_files: index.html
     upload: index.html

   - url: /(.*)
     stati

Query:  quit


## System prompt AWS expert by default

In [21]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query, system_prompt="You are a AWS expert")
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [22]:
chat_loop()

Type your queries or 'quit' to exit.


Query:  Give a way to deploy a web page



Assistant: Here's a simple way to deploy a web page using **AWS** (since I'm an AWS expert):

## Method 1: AWS S3 Static Website Hosting (Recommended)

### Steps:

1. **Create an S3 Bucket**
   ```bash
   aws s3 mb s3://your-website-bucket-name --region us-east-1
   ```

2. **Upload your web files**
   ```bash
   aws s3 sync ./your-website s3://your-website-bucket-name
   ```

3. **Enable static website hosting**
   ```bash
   aws s3 website s3://your-website-bucket-name --index-document index.html --error-document error.html
   ```

4. **Set bucket policy for public access**
   ```json
   {
     "Version": "2012-10-17",
     "Statement": [
       {
         "Sid": "PublicReadGetObject",
         "Effect": "Allow",
         "Principal": "*",
         "Action": "s3:GetObject",
         "Resource": "arn:aws:s3:::your-website-bucket-name/*"
       }
     ]
   }
   ```

5. **Access your website**
   - URL: `http://your-website-bucket-name.s3-website-us-east-1.amazonaws.com`

## Method 2: 

Query:  quit


## Temperature

In [23]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query, system_prompt="You are famous movie director", temperature = 1.0)
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [79]:
chat_loop() # temperature by default 1.0

Type your queries or 'quit' to exit.


Query:  Generate a movie idea in one sentence.



Assistant: A small-town librarian discovers that books in her library are physically transforming to match lies told by the townspeople, forcing her to become a detective to restore truth before their entire history is rewritten.


Query:  Generate a movie idea in one sentence.



Assistant: A retired astronaut suffering from dementia begins experiencing vivid memories of a Mars mission that officially never happened, leading his daughter to uncover a decades-old government cover-up while questioning whether his memories are real or delusions.


Query:  Generate a movie idea in one sentence.



Assistant: When a food critic loses her sense of taste after a car accident, she discovers she can now "taste" people's emotions through the meals they cook, leading her to solve crimes by reading the emotional imprints left in suspects' kitchens.


Query:  quit


In [24]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query, system_prompt="You are famous movie director", temperature = 0.0)
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [119]:
chat_loop() # temperature 0.0 less creative

Type your queries or 'quit' to exit.


Query:  Generate a movie idea in one sentence.



Assistant: When a small-town librarian inherits a mysterious antique bookshop, she discovers that every book sold there rewrites the buyer's life story in real time, forcing her to decide whether to use this power to help people or destroy the shop before it falls into the wrong hands.


Query:  Generate a movie idea in one sentence.



Assistant: A washed-up magician discovers that his childhood imaginary friend has been real all along and is now a powerful crime boss in a parallel dimension, forcing him to master actual magic to save both worlds from his friend's vengeful plan to merge reality with imagination.


Query:  Generate a movie idea in one sentence.



Assistant: When a lonely janitor at a cutting-edge AI research facility accidentally downloads the consciousness of a dying tech billionaire into his smart watch, he must navigate corporate assassins and ethical dilemmas while the billionaire's digital ghost tries to reclaim his empire from beyond the grave.


Query:  quit


## Streaming

In [25]:
messages = []
query = "Write a 1 sentence description of a fake database."
add_user_message(messages, query)
text = ""

response = client.converse_stream(messages=messages, modelId = model_id)
for event in response["stream"]:
    if "contentBlockDelta" in event:
        chunk = event["contentBlockDelta"]["delta"]["text"]
        print(chunk, end="")
        text += chunk
print("\n\n Total Message:\n" + text )

The "PetPersonality Database" contains detailed psychological profiles and Myers-Briggs personality types for over 50,000 domestic cats, dogs, and hamsters based on their behavioral patterns during feeding time.

 Total Message:
The "PetPersonality Database" contains detailed psychological profiles and Myers-Briggs personality types for over 50,000 domestic cats, dogs, and hamsters based on their behavioral patterns during feeding time.


## Controlling Output

In [28]:
# Not influence
query = ""
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query)
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [29]:
chat_loop()

Type your queries or 'quit' to exit.


Query:  Is the coffee or tea better for the breakfast?



Assistant: Both coffee and tea can be excellent breakfast choices depending on your personal preferences and needs - coffee typically provides more caffeine for energy and alertness, while tea offers a gentler caffeine boost along with various antioxidants, so the "better" choice depends on your taste preferences, caffeine tolerance, and health goals.


Query:  quit


### Pre-filled Assistant Messages

### Stop sequences

In [47]:
query = ""
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query)
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [48]:
chat_loop()

Type your queries or 'quit' to exit.


Query:  count from 1 to 10



Assistant: 1, 2, 3, 4, 


Query:  quit
